In [ ]:
import numpy as np
import argparse
from scipy import io, spatial
import time
from random import shuffle
import random
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix
import scipy.io as sio
import scipy.io as sio
import os
import shutil



rand_seed = 42
dataset = '/content/DEFECT'
lr = 0.1
norm_type = 'L2'
early_stop = 20
epochs = 20

def normalizeFeature(x):
    # x = N x d (d:feature dimension, N:number of instances)
    x = x + 1e-10
    feature_norm = np.sum(x**2, axis=1)**0.5 # l2-norm
    feat = x / feature_norm[:, np.newaxis]

    return feat

def update_W(W, idx, train_classes, beta):

    for j in idx:

        X_n = X_train[:, j]
        y_n = labels_train[j]
        y_ = train_classes[train_classes!=y_n]
        XW = np.dot(X_n, W)
        gt_class_score = np.dot(XW, train_sig[:, y_n])

        for i in range(len(y_)):
            label = random.choice(y_)
            score = 1+np.dot(XW, train_sig[:, label])-gt_class_score # acc. to original paper, margin shd always be 1.
            if score>0:
                Y = np.expand_dims(train_sig[:, y_n]-train_sig[:, label], axis=0)
                W += lr*beta[int(y_.shape[0]/(i+1))]*np.dot(np.expand_dims(X_n, axis=1), Y)
                break
    return W

def fit():

    print('Training...\n')

    best_val_acc = 0.0
    best_tr_acc = 0.0
    best_val_ep = -1
    best_tr_ep = -1

    rand_idx = np.arange(X_train.shape[1])

    W = np.random.rand(X_train.shape[0], train_sig.shape[0])
    W = normalizeFeature(W.T).T

    train_classes = np.unique(labels_train)

    beta = np.zeros(len(train_classes))
    for i in range(1, beta.shape[0]):
        sum_alpha=0.0
        for j in range(1, i+1):
            sum_alpha+=1/j
        beta[i] = sum_alpha

    for ep in range(epochs):

        start = time.time()

        shuffle(rand_idx)

        W = update_W(W, rand_idx, train_classes, beta)

        tr_acc = zsl_acc(X_train, W, labels_train, train_sig)
        val_acc = zsl_acc(X_val, W, labels_val, val_sig)

        end = time.time()

        elapsed = end-start

        print('Epoch:{}; Train Acc:{}; Val Acc:{}; Time taken:{:.0f}m {:.0f}s\n'.format(ep+1, tr_acc, val_acc, elapsed//60, elapsed%60))

        if val_acc>best_val_acc:
            best_val_acc = val_acc
            best_val_ep = ep+1
            best_W = np.copy(W)

        if tr_acc>best_tr_acc:
            best_tr_ep = ep+1
            best_tr_acc = tr_acc

        if ep+1-best_val_ep>early_stop:
            print('Early Stopping by {} epochs. Exiting...'.format(epochs-(ep+1)))
            break

    print('\nBest Val Acc:{} @ Epoch {}. Best Train Acc:{} @ Epoch {}\n'.format(best_val_acc, best_val_ep, best_tr_acc, best_tr_ep))

    return best_W

def zsl_acc(X, W, y_true, sig): # Class Averaged Top-1 Accuarcy

    XW = np.dot(X.T, W)# N x k
    dist = 1-spatial.distance.cdist(XW, sig.T, 'cosine')# N x C(no. of classes)
    predicted_classes = np.array([np.argmax(output) for output in dist])
    cm = confusion_matrix(y_true, predicted_classes)
    print(y_true)
    print(predicted_classes)
    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    acc = sum(cm.diagonal())/sig.shape[1]

    return acc

def evaluate():

    best_W = fit()

    print('Testing...\n')

    test_acc = zsl_acc(X_test, best_W, labels_test, test_sig)

    print('Test Acc:{}'.format(test_acc))

random.seed(rand_seed)
np.random.seed(rand_seed)

data_folder = dataset+'/'
res101 = io.loadmat(data_folder+'res101.mat')
att_splits=io.loadmat(data_folder+'att_splits.mat')

train_loc = 'train_loc'
val_loc = 'val_loc'
test_loc = 'test_unseen_loc'

feat = res101['features']
# Shape -> (dxN)
X_train = feat[:, np.squeeze(att_splits[train_loc]-1)]
X_val = feat[:, np.squeeze(att_splits[val_loc]-1)]
X_test = feat[:, np.squeeze(att_splits[test_loc]-1)]

print('Tr:{}; Val:{}; Ts:{}\n'.format(X_train.shape[1], X_val.shape[1], X_test.shape[1]))

labels = res101['labels']
labels_train = np.squeeze(labels[np.squeeze(att_splits[train_loc]-1)])
labels_val = np.squeeze(labels[np.squeeze(att_splits[val_loc]-1)])
labels_test = np.squeeze(labels[np.squeeze(att_splits[test_loc]-1)])

train_labels_seen = np.unique(labels_train)
val_labels_unseen = np.unique(labels_val)
test_labels_unseen = np.unique(labels_test)

i=0
for labels in train_labels_seen:
    labels_train[labels_train == labels] = i
    i+=1

j=0
for labels in val_labels_unseen:
    labels_val[labels_val == labels] = j
    j+=1

k=0
for labels in test_labels_unseen:
    labels_test[labels_test == labels] = k
    k+=1




# Shape -> (Number of attributes, Number of Classes)
sig = att_splits['att']
train_sig = sig[:, train_labels_seen-1]
val_sig = sig[:, val_labels_unseen-1]
test_sig = sig[:, test_labels_unseen-1]

if norm_type=='std':
    print('Standard Scaling\n')
    scaler = preprocessing.StandardScaler()
    scaler.fit(X_train.T)

    X_train = scaler.transform(X_train.T).T
    X_val = scaler.transform(X_val.T).T
    X_test = scaler.transform(X_test.T).T

if norm_type=='L2':
    print('L2 norm Scaling\n')
    X_train = normalizeFeature(X_train.T).T
    # X_val = normalizeFeature(X_val.T).T
    # X_test = normalizeFeature(X_test.T).T

evaluate()

